## Coronavirus Cases and Social Distancing Scores in Southern California


#### Alex Ehrens
#### ESM267 Asst. 3

In the ArcGIS Living Atlas, I have found two data layers: one with the number of confirmed Covid-19 cases by county and one with a "social distancing" score by county. The social distancing score comes from Unacast and assigns counties a grade based on the percent change in average distance travelled and visitation to non-essential venues relative to before Covid. Better grades are given to counties where there has been a larger decrease in travel and non-essential visits. I have displayed information from both layers for counties in Southern California.

In [1]:
# store layer IDs
cases_service_id = "628578697fb24d8ea4c32fa0c5ae1843"
scores_service_id = "ab72fb3e9bf24d9594f0b942718bffeb"

In [2]:
# import GIS module
from arcgis.gis import GIS

# import other modules
from arcgis.features import FeatureLayer

In [3]:
# using anonymous server to make life easier for Frew
my_gis = GIS()

Now it's time to start putting my map together:

In [4]:
# start with default base map
my_map = my_gis.map('Los Angeles, California', zoomlevel = 7)

Gather the two data layers from publicly available content:

In [5]:
# add social distancing score layer
scores = my_gis.content.get(scores_service_id)
scores

<Item title:"Unacast Social Distancing Score (Latest Available)" type:Feature Layer Collection owner:Unacast_Partner>

In [6]:
# add Covid cases layer 
cases = my_gis.content.get(cases_service_id)
cases

<Item title:"COVID-19 Cases US" type:Feature Layer Collection owner:CSSE_GISandData>

Now that I have the layers that I'm interested in, I want to also add a layer outlining the counties of interest for this map (which are Southern California counties). To do that I need to query the attributes in one of the layers, selecting only those that have names matching the names of Southern California counties. But apparently the data I added up above was a Feature Service, which can't be queried. So I needed to find and upload the Feature Layer of one of the layers. I used the Unacast social distance score layer for this because I checked in its documentation that it had county_name as a field.

In [7]:
# find feature layer url
scores_layer_url = scores.layers
scores_layer_url

[<FeatureLayer url:"https://services.arcgis.com/DO4gTjwJVIJ7O9Ca/arcgis/rest/services/Unacast_Latest_Available__Visitation_and_Distance_/FeatureServer/0">]

In [8]:
# use feature url to upload as feature layer for scores data (because it has county_name field to query)
scores_lyr_url = 'https://services.arcgis.com/DO4gTjwJVIJ7O9Ca/arcgis/rest/services/Unacast_Latest_Available__Visitation_and_Distance_/FeatureServer/0'

scores_layer = FeatureLayer(scores_lyr_url)
scores_layer

<FeatureLayer url:"https://services.arcgis.com/DO4gTjwJVIJ7O9Ca/arcgis/rest/services/Unacast_Latest_Available__Visitation_and_Distance_/FeatureServer/0">

In [9]:
# just double checking the list of field names
for f in scores_layer.properties.fields:
    print(f['name'])

OBJECTID
state_code
state_fips
state_name
county_fips
county_name
county_population
last_updated
localeventdate
weekday
covid
grade_total
n_grade_total
daily_distance_diff
grade_distance
n_grade_distance
daily_visitation_diff
grade_visitation
n_grade_visitation
encounters_rate
grade_encounters
n_grade_encounters
county_fips_int
Shape__Area
Shape__Length


In [10]:
# query features to get an outline of only Southern CA counties
southern_ca_scores = scores_layer.query(where = "county_name = 'San Luis Obispo County' OR county_name = 'Kern County' OR county_name = 'San Bernardino County' OR county_name = 'Santa Barbara County' OR county_name = 'Ventura County' OR county_name = 'Los Angeles County' OR county_name = 'Orange County' OR county_name = 'Riverside County' OR county_name = 'Imperial County' OR county_name = 'San Diego County'")

### My Map: Social distancing scores and Covid-19 cases in Southern California

In [11]:
# add my layers to my map
my_map.add_layer(southern_ca_scores)
my_map

MapView(layout=Layout(height='400px', width='100%'))

In [12]:
# had to add this layer in here again to order it properly
my_map.add_layer(scores)
my_map.add_layer(southern_ca_scores)
my_map.add_layer(cases)

Based on this map, it appears that Santa Barbara and San Luis Obispo Counties have been social distancing better than any other Southern California county. This means that residents of those counties have done better at reducing travel distance and visitations to non-essential venues. As a result, the amount of confirmed Covid-19 cases is relatively smaller in those counties, shown by the size of the red circle. Orange County has been the worst at social distancing, and Los Angeles and San Diego counties have also been bad. As a result, the number of confirmed cases are higher in those counties. Unfortunately, the Covid-19 cases data layer was missing data for Los Angeles County, as that would likely be the highest of any of the counties. This map goes to show that social distancing guidelines actually do help limit the spread of Covid-19 and more people should actually follow them! 